In [1]:
import pandas as pd
import numpy as np
import random
from joblib import dump, load
import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from ds_code.function.utils import sliding_window
from ds_code.function.models import *

## Data preparation and preprocessing

##### Extra attributes for distinction between provinces

In [2]:
city_data = pd.read_csv("data/region/vietnam/extra_info.csv", index_col=0)
city_data

,lat,lng,population
id,,,
1704774326,10.7756,106.7019,1.513600e+07
1704413791,21.0000,105.8500,8.246600e+06
1704000623,20.8651,106.6838,2.103500e+06
1704783472,10.0333,105.7833,1.237300e+06
1704863046,10.9500,106.8167,1.104000e+06
...,...,...,...
1704495953,22.8333,104.9833,5.555900e+04
1704000217,22.1333,105.8333,4.503600e+04
1704983526,22.3992,103.4392,4.297300e+04


##### Data preprocessing

In [3]:
weather_np = []
air_np = []
init_np = []

# take some minutes to run
for city_id in city_data.index:        
    # load air quality and weather data files 
    air_df = pd.read_csv("data/air_quality/" + str(city_id) + ".csv")
    weather_df = pd.read_csv("data/weather/" + str(city_id) + ".csv")
    
    # air quality data preprocessing
    air_df = air_df.loc[(air_df.iloc[:, 1:] >= 0).all(axis=1)]
    air_df.drop("aqi", axis=1, inplace=True)
    air_df.reset_index(drop=True, inplace=True)

    # weather data preprocessing
    weather_df.dropna(axis=0, inplace=True)
    weather_df.reset_index(drop=True, inplace=True)

    # convert wind direction attribute
    weather_df["wind_x_component"] = np.cos(weather_df["wind_direction_10m"] / (180 / np.pi))
    weather_df["wind_y_component"] = np.sin(weather_df["wind_direction_10m"] / (180 / np.pi))
    weather_df.drop("wind_direction_10m", axis=1, inplace=True)
    
    # making sliding windows
    X, y = sliding_window(weather_df, air_df, target_size="same")
    
    # extra attibutes
    m = X.shape[0]
    extra_attr = city_data.loc[city_id]
    lat = np.full((m, 1), extra_attr[0])
    lng = np.full((m, 1), extra_attr[1])
    population = np.full((m, 1), extra_attr[2])
    init_data = np.hstack((lat, lng, population))
    
    weather_np.append(X)
    air_np.append(y)
    init_np.append(init_data)
    
weather_np = np.vstack(weather_np)
air_np = np.vstack(air_np)
init_np = np.vstack(init_np)

In [ ]:
#dump((weather_np, air_np, init_np), "gru_raw_dataset.pkl")

['gru_raw_dataset.pkl']

In [2]:
#weather_np, air_np, init_np = load("gru_raw_dataset.pkl")

## Modelling

##### Scaler for normalizing and revert data to original

In [6]:
class StandardScaler(torch.nn.Module):
    def __init__(self):
        super(StandardScaler, self).__init__()
        self.mean = None
        self.std = None

    def fit(self, X):
        self.mean = X.mean(dim=0, keepdim=True)
        self.std = X.std(dim=0, keepdim=True)

    def forward(self, X):
        return (X - self.mean) / self.std

    def inverse_transform(self, X_scaled):
        return X_scaled * self.std + self.mean

##### GRU model

In [4]:
class CustomGRU(nn.Module):
    def __init__(self, input_size, output_size, seq_len=4, label_scaler=None):
        super(CustomGRU, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.seq_len = seq_len
        self.label_scaler = label_scaler
        
        # fully connected layers to generate initial hidden state for GRU layers
        self.init_nn = nn.Sequential(
            nn.LayerNorm(3),
            nn.Linear(3, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU()
        )
        
        # GRU layers
        self.flatten = nn.Flatten(1, -1)
        self.normalize = nn.LayerNorm(input_size * seq_len)
        self.gru1 = nn.GRU(input_size, 256, batch_first=True)
        self.gru2 = nn.GRU(256, 128, batch_first=True)
        self.gru3 = nn.GRU(128, 64, batch_first=True)
        
        # Final fully connected layer
        self.linear = nn.Linear(64, output_size)

    def forward(self, inp, rescale=False):
        X, init_data = inp
        X = self.flatten(X)
        X = self.normalize(X).reshape((-1, self.seq_len, self.input_size))
        init_data = self.init_nn(init_data.unsqueeze(0))
        X, _ = self.gru1(X, init_data)
        X, _ = self.gru2(X)
        X, _ = self.gru3(X)
        X = self.linear(X)
        # Rescale if needed with a standard scaler (for actual prediction)
        if rescale:
            X = self.label_scaler.inverse_transform(X)
        return X
    
    def predict(self, inp, numpy_output=True):
        X, init_data = inp
        inp = (Tensor(X), Tensor(init_data))
        self.eval()
        with torch.no_grad():
            output = self(inp, rescale=True)
        if numpy_output:
            output = output.numpy()
        return output[:, -1]

## Prepare for training

##### Create dataset

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y, init_data, label_scaler=None):
        self.X = Tensor(X)
        self.y = Tensor(y)
        self.init_data = Tensor(init_data)
        
        if label_scaler is not None:
            self.scaler = label_scaler
            self.scaler.fit(self.y)
            self.y = label_scaler(self.y)
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, index):
        return (self.X[index], self.init_data[index]), self.y[index]

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##### Training function

In [64]:
def train_model(model, train_dataloader, valid_dataloader, learning_rate=1e-3, num_epochs=10):
    optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=0)
    scheduler = ReduceLROnPlateau(optimizer, patience=5)
    criterion = nn.MSELoss()
    val_patience = 20
    waited_epoch = 0
    best_val_loss = 9999
    
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Learning rate: {optimizer.param_groups[0]['lr']}")
        for i, (X, y) in enumerate(train_dataloader):
            X = X[0].to(device), X[1].to(device)
            y = y.to(device)
            
            # Zero gradients
            optimizer.zero_grad()
            # Forward pass
            output = model(X)

            # Compute loss
            loss = criterion(output, y)
            total_loss += loss

            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            if (i + 1) % 1000 == 0:
                print(f"Batch {i + 1}:", f"Loss: {total_loss / i:.4f}")

        print(f"Training loss: {total_loss / len(train_dataloader):.4f}")
        
        with torch.no_grad():
            total_loss = 0
            for X, y in valid_dataloader:
                X = X[0].to(device), X[1].to(device)
                y = y.to(device)
                
                # Forward pass
                output = model(X)

                # Compute loss
                loss = criterion(output[:, -1], y[:, -1])
                total_loss += loss
            scheduler.step(total_loss)
            
        print(f"Validaton loss: {total_loss / len(valid_dataloader):.4f}\n")
            
        if total_loss < best_val_loss:
            waited_epoch = 0
            best_val_loss = total_loss
            torch.save(model, "models/gru.pth")
        else:
            waited_epoch += 1
            if waited_epoch == val_patience:
                print("Training stopped.")
                return

##### Reduce data size for memory efficiency

In [4]:
weather_np = weather_np.astype("float32")
air_np = air_np.astype("float32")
init_np = init_np.astype("float32")

##### Splitting the dataset

In [5]:
random.seed(42)
idx = [i for i in range(len(weather_np))]
random.shuffle(idx)
train_idx, test_idx = idx[:1800000], idx[1800000:]
X_train, X_test, y_train, y_test, init_train, init_test = weather_np[train_idx], weather_np[test_idx], air_np[train_idx], air_np[test_idx], init_np[train_idx], init_np[test_idx]

##### Prepare training and validation dataset

In [67]:
train_scaler = StandardScaler()
valid_scaler = StandardScaler()
train_dataset = TimeSeriesDataset(X_train[:1700000], y_train[:1700000], init_train[:1700000], train_scaler)
valid_dataset = TimeSeriesDataset(X_train[1700000:], y_train[1700000:], init_train[1700000:], valid_scaler)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

##### Create model

In [68]:
rnn = CustomGRU(input_size=9, output_size=6, seq_len=4, label_scaler=train_scaler)
rnn.to(device)

CustomGRU(
  (label_scaler): StandardScaler()
  (init_nn): Sequential(
    (0): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
    (1): Linear(in_features=3, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=256, bias=True)
    (4): ReLU()
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (normalize): LayerNorm((36,), eps=1e-05, elementwise_affine=True)
  (gru1): GRU(9, 256, batch_first=True)
  (gru2): GRU(256, 128, batch_first=True)
  (gru3): GRU(128, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=6, bias=True)
)

## Training

In [ ]:
# output is for illustraion purpose
train_model(rnn, train_dataloader, valid_dataloader, learning_rate=1e-3, num_epochs=200)

Epoch 1/200
Learning rate: 0.001
Training loss: 1.0024
Validaton loss: 0.9818

Epoch 2/200
Learning rate: 0.001
Training loss: 0.9957
Validaton loss: 0.9784

Epoch 3/200
Learning rate: 0.001
Training loss: 1.0086
Validaton loss: 0.9665

Epoch 4/200
Learning rate: 0.001
Training loss: 0.9717
Validaton loss: 0.9831

Epoch 5/200
Learning rate: 0.001
Training loss: 0.9665
Validaton loss: 0.9327

Epoch 6/200
Learning rate: 0.001
Training loss: 0.9364
Validaton loss: 0.9256

Epoch 7/200
Learning rate: 0.001
Training loss: 0.9351
Validaton loss: 0.9331

Epoch 8/200
Learning rate: 0.001
Training loss: 0.9300
Validaton loss: 0.9721

Epoch 9/200
Learning rate: 0.001
Training loss: 0.9270
Validaton loss: 0.9240

Epoch 10/200
Learning rate: 0.001
Training loss: 0.9205
Validaton loss: 0.9169

Epoch 11/200
Learning rate: 0.001
Training loss: 0.9220
Validaton loss: 0.9308

Epoch 12/200
Learning rate: 0.001
Training loss: 0.9122
Validaton loss: 0.9427

Epoch 13/200
Learning rate: 0.001
Training loss: 

## Evaluating the model

In [6]:
model = torch.load("models/gru.pth", map_location="cpu")

C:\Users\trann\AppData\Local\Temp\ipykernel_25456\2437652950.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("models/gru.pth", map_location="cpu")


##### Root mean squared error

In [7]:
pd.Series(root_mean_squared_error(model.predict((X_test, init_test)), y_test[:, -1], multioutput="raw_values"), 
          index=["co", "no2", "o3", "so2", "pm2_5", "pm10"])

co       467.083984
no2       11.538258
o3        24.473648
so2        8.739453
pm2_5     34.836983
pm10      39.177345
dtype: float32

##### Mean absolute error

In [8]:
pd.Series(mean_absolute_error(model.predict((X_test, init_test)), y_test[:, -1], multioutput="raw_values"), 
          index=["co", "no2", "o3", "so2", "pm2_5", "pm10"])

co       253.260666
no2        7.176695
o3        17.367544
so2        4.860417
pm2_5     20.831501
pm10      23.449749
dtype: float32